# Classificador de Twitter

### Autor: Ricardo Robson ~rrms at cin.ufpe.br
### Baseado em: http://minerandodados.com.br/index.php/2017/03/15/analise-de-sentimentos-twitter-como-fazer/

O objetivo deste notebook é construir um classificador de sentimento de tweets, utilizando o Algoritmo Naive Bayes, classificando textos de tweets em Positivo ou Negativo.

Faremos uso de um cospus contendo pouco mais de 2900 tweets já classificados.

In [96]:
#Imports que serão usados ao longo da construção e análise do classificador
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from nltk.stem.snowball import SnowballStemmer
from googletrans import Translator

In [97]:
#Lendo o corpus de tweets classificados
dataset = pd.read_csv('twitter_corpus.csv')
dataset.count()

Text         4088
Sentiment    4088
dtype: int64

In [98]:
#Separando textos e classes (sentimentos)
tweets = dataset['Text'].values
classes = dataset['Sentiment'].values
classes

array(['neg', 'pos', 'pos', ..., 'pos', 'pos', 'pos'], dtype=object)

#### Treinando o modelo

In [99]:
#Criamos um objeto do tipo CountVectorizer chamado vectorizer.
vectorizer = CountVectorizer(analyzer="word")

#Usamos o objeto vectorizer para calcular a frequência de todas as palavras 
#da lista de tweets e armazenamos seu retorno em freq_tweets.
freq_tweets = vectorizer.fit_transform(tweets)

#Criamos um objeto chamado modelo do tipo Naive Bayes Multinomial.
#Esse é o algoritmo de classificação que iremos utilizar.
modelo = MultinomialNB()

#Por fim treinamos o modelo usando a frequência de palavras (freq_tweets)
#e as classes de cada instância.
modelo.fit(freq_tweets,classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

#### Treinando o modelo com stop words

In [100]:
stopwords_vectorizer = CountVectorizer(analyzer="word", stop_words='english')
stopwords_freq_tweets = stopwords_vectorizer.fit_transform(tweets)
stopwords_modelo = MultinomialNB()
stopwords_modelo.fit(stopwords_freq_tweets,classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

#### Testando modelo simples

In [101]:
#Criando uma pequena lista para testes
testes = ['Feeeling like shit right now. I really want to sleep, but nooo I have 3 hours of dancing and an art assignment to finish.',
         'goodbye exams, HELLO ALCOHOL TONIGHT',
         'trae is so sweet! He just bought me a new baithing suit!! Wove him']
#Testando
freq_testes = vectorizer.transform(testes)
modelo.predict(freq_testes)

array(['neg', 'pos', 'pos'], 
      dtype='<U3')

#### Testando modelo com stopwords

In [102]:
freq_testes = stopwords_vectorizer.transform(testes)
stopwords_modelo.predict(freq_testes)

array(['neg', 'neg', 'pos'], 
      dtype='<U3')

#### Treinando modelo com stemming

In [103]:
stem_modelo = MultinomialNB()
stemmer = SnowballStemmer("english")
stem_vectorizer = CountVectorizer(stemmer)
stemeed_tweets = []
for tweet in tweets:
    stemeed_tweets.append(' '.join([stemmer.stem(word) for word in tweet.split()]))

freq_steemed_tweets = stem_vectorizer.fit_transform(stemeed_tweets)
stem_modelo.fit(freq_steemed_tweets,classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

#### Testando modelo com stemming

In [104]:
stem_testes = []
for test in testes:
    stem_testes.append(' '.join([stemmer.stem(word) for word in test.split()]))

stem_freq_testes = stem_vectorizer.transform(stem_testes)
stem_modelo.predict(stem_freq_testes)

array(['neg', 'pos', 'pos'], 
      dtype='<U3')

#### Treinando modelo com stopwords e stemming

In [105]:
stem_stopwords_vectorizer = CountVectorizer(analyzer="word", stop_words='english')
stem_stopwords_freq_tweets = stem_stopwords_vectorizer.fit_transform(stemeed_tweets)
stem_stopwords_modelo = MultinomialNB()
stem_stopwords_modelo.fit(stem_stopwords_freq_tweets,classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

#### Testando modelo com stopwords e stemming

In [106]:
stem_stopwords_freq_testes = stem_stopwords_vectorizer.transform(stem_testes)
stem_stopwords_modelo.predict(stem_stopwords_freq_testes)

array(['neg', 'pos', 'pos'], 
      dtype='<U3')

#### Função que recupera os tweets de um CSV e retorna a lista de tweets já traduzidos para o inglês

In [107]:
def translateTweers(file_name, sep_name):
    print("Traduzindo '"+file_name+"'")
    tweets_csv = pd.read_csv(file_name, sep=sep_name)
    translator = Translator()
    tweets_list = []
    dates = []
    for index, tweet in enumerate(tweets_csv['tweet']):
        print('.', end="")
        try:
            translated = translator.translate(tweet, dest='en')
            tweets_list.append(translated.text)
            dates.append(tweets_csv['date'][index])
        except ValueError:
            print("ERRO ao tentar traduzir em '"+file_name);
    return dates, tweets_list

### Predição do sentimento para cada time durante o campeonato

In [108]:
teams = ['FNC', 'MSF', 'C9', 'LZ','WE', 'RNG', 'SKT', 'SSG']

tweets_dates = []
tweets_list = []

for team in teams:

    dates, tranlated_tweets = translateTweers(team,";")
    tweets_dates.extend(dates)
    tweets_list.extend(tranlated_tweets)
    
tweets_list_vector = vectorizer.transform(tweets_list)
result = modelo.predict(tweets_list_vector)

stp_tweets_list_vector = stopwords_vectorizer.transform(tweets_list)
stp_result = stopwords_modelo.predict(stp_tweets_list_vector)

stem_teams = []
for tweet in tweets_list:
    stem_teams.append(' '.join([stemmer.stem(word) for word in tweet.split()]))
stem_freq = stem_vectorizer.transform(stem_teams)
stem_result = stem_modelo.predict(stem_freq)

stem_stop_freq = stem_stopwords_vectorizer.transform(stem_teams)
stem_stp_result = stem_stopwords_modelo.predict(stem_stop_freq)

print(str(len(tweets_list))+"\t-\t"+str(len(result)))

Traduzindo 'FNC'
............................................................................................................................................................................................................................................................................................................Traduzindo 'MSF'
............................................................................................................................................................................................................................................................................................................Traduzindo 'C9'
............................................................................................................................................................................................................................................................................................................Traduzindo 'LZ'
..................................

In [109]:
print(len(result))
percentual_positives = len(result[result == 'pos'])/len(result)
percentual_negatives = 1-percentual_positives
print("Pos: "+str(percentual_positives)+" Neg:"+str(percentual_negatives))

7398
Pos: 0.22127602054609355 Neg:0.7787239794539065


#### Escrevendo CSV com os resultados

In [110]:
f = open('tweets_classifieds.csv','w')
for index, tweet_result in enumerate(result):
    text = tweets_dates[index]+';'+tweets_list[index]+';'+tweet_result+';'+stp_result[index]+';'+stem_result[index]+';'+stem_stp_result[index]+'\n'
    f.write(text)
    
f.close()
print("Finished")

Finished
